In [1]:
####### Imports #######
import re
import os
import sys
sys.path.append('/home/pfa/Documents/PathologicalGaitClassification')
import numpy as np
from keras.applications import Xception
from keras.models import Model
from keras.layers import Dense, Input, ConvLSTM2D, GlobalAveragePooling2D, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import Nadam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing import image
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from sort import sort_nicely
from multiprocessing import Process, Manager
import pickle
import json

Using TensorFlow backend.


In [2]:
## Get datase
base_dir = '/home/datasets/GAIT-IT'
metadata_dir = '/home/datasets/metadata'

sample_count = 0
all_images = {'train': [], 'validation': [], 'test': []}
all_labels = {'train': [], 'validation': [], 'test': []}
subjects_data = {}
train_subjs = ['s1','s2','s3','s4','s5','s6','s7','s8','s9','s10','s11','s12','s13','s14','s15','s16','s17','s18','s19']
validation_subjs = ['s20','s21']
test_subjs = ['s22','s23']
train_images = []; train_labels = []
validation_images = []; validation_labels = []
test_images = []; test_labels = []

classes = {'Diplegic' : 0, 'Hemiplegic' : 1, 'Neuropathic' : 2, 'Normal' : 3, 'Parkinson' : 4}
classes_inv = {0 : 'Diplegic', 1 : 'Hemiplegic', 2 : 'Neuropathic', 3 : 'Normal', 4 : 'Parkinson'}

In [3]:
## Train silhouettes directories

# Sort pathologies, OCD purposes only
pathologies = list(classes.keys())
sort_nicely(pathologies)
for pathology in pathologies:

    pathology_dir = base_dir + '/{}'.format(pathology)
    print(pathology_dir)
    pathology_subj_folders = [name for name in os.listdir(pathology_dir) if os.path.isdir(os.path.join(pathology_dir, name))]
    sort_nicely(pathology_subj_folders)
    
    # /Pathology/subj{i}/silhouettes/subj_{i}-pat_{j}-lvl_{k}-{l}_{direction}
    for subj_folder in pathology_subj_folders:

        # if subj_folder not in ['s1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12','s13','s14']: continue
        if subj_folder in train_subjs: subj_set = 'train'
        elif subj_folder in validation_subjs: subj_set = 'validation'
        elif subj_folder in test_subjs: subj_set = 'test'

        subj_folder_dir = os.path.join(pathology_dir, subj_folder)
        # print(subj_folder_dir)
        subj_silhouettes_dir = os.path.join(subj_folder_dir, 'silhouettes', 'side_view')
        # print(subj_silhouettes_dir)

        subj_silhouettes_folders = [name for name in os.listdir(subj_silhouettes_dir) if os.path.isdir(os.path.join(subj_silhouettes_dir, name))]
        sort_nicely(subj_silhouettes_folders)

        folders = [f for f in subj_silhouettes_folders if '_' in f]
        for folder in folders:

            # Initialize dictionary to store key frames
            key_frames = {}

            # Directory with metadata about current folder
            subj_silhouettes_metadata_dir = subj_silhouettes_dir.replace('GAIT-IT', 'metadata')
            subj_pat_metadata = os.path.join(subj_silhouettes_metadata_dir,'metadata/key_frames.json')

            with open(subj_pat_metadata) as f:
                key_frames = json.load(f)

            # Directory with the sillouettes images
            subj_pat_lvl_dir = os.path.join(subj_silhouettes_dir, folder)
            # print(subj_pat_lvl_dir)

            files = os.listdir(subj_pat_lvl_dir)
            sort_nicely(files)
            file_names = [files[f] for f in key_frames[folder]]
            
            # Convert images to numpy arrays, put in batches
            # for file_name in file_names:
            for i in range(0, len(file_names)-8, 9):
                for j in range(0,9):
                    file_path = os.path.join(subj_pat_lvl_dir, file_names[i+j])
                    img = image.load_img(file_path, target_size=(224, 224))
                    img_tensor = image.img_to_array(img)
                    sample_count += 1

                    all_images[subj_set].append(img_tensor)
                    all_labels[subj_set].append(classes[pathology])
print(len(all_images['train'])); print(len(all_images['validation'])); print(len(all_images['test']))
print(len(all_labels['train'])); print(len(all_labels['validation'])); print(len(all_labels['test']))
                

/home/datasets/GAIT-IT/Diplegic
/home/datasets/GAIT-IT/Hemiplegic
/home/datasets/GAIT-IT/Neuropathic
/home/datasets/GAIT-IT/Normal
/home/datasets/GAIT-IT/Parkinson
27486
3564
3285
27486
3564
3285


In [4]:
img_height = 224
img_width = 224
no_of_frames = 9
channels = 3
no_of_epochs = 50
batch_size_value = 5

cnn_base = Xception(input_shape=(img_width, img_height, channels), weights="imagenet", include_top=False)
cnn_base.trainable = False

cnn_base.summary()

all_labels_array = {'train': [], 'validation': [], 'test': []}
all_features_array = {'train': [], 'validation': [], 'test': []}

for subj_set in all_images:
    print(subj_set)
    for i in range(0,len(all_images[subj_set])-8,9):
        features_array = np.stack(all_images[subj_set][i:i+9])
        predictions = cnn_base.predict(features_array)
        all_labels_array[subj_set].append(all_labels[subj_set][i])
        all_features_array[subj_set].append(predictions)

    all_features_array[subj_set] = np.array(all_features_array[subj_set])
    all_labels_array[subj_set] = np.array(all_labels_array[subj_set])
    all_labels_array[subj_set] = to_categorical(all_labels_array[subj_set])

    print(all_features_array[subj_set].shape)
    print(all_labels_array[subj_set].shape)

_______
block5_sepconv1 (SeparableConv2 (None, 14, 14, 728)  536536      block5_sepconv1_act[0][0]        
__________________________________________________________________________________________________
block5_sepconv1_bn (BatchNormal (None, 14, 14, 728)  2912        block5_sepconv1[0][0]            
__________________________________________________________________________________________________
block5_sepconv2_act (Activation (None, 14, 14, 728)  0           block5_sepconv1_bn[0][0]         
__________________________________________________________________________________________________
block5_sepconv2 (SeparableConv2 (None, 14, 14, 728)  536536      block5_sepconv2_act[0][0]        
__________________________________________________________________________________________________
block5_sepconv2_bn (BatchNormal (None, 14, 14, 728)  2912        block5_sepconv2[0][0]            
__________________________________________________________________________________________________
bl

In [6]:
img_height = 7
img_width = 7
no_of_frames = 9
channels = 2048
no_of_epochs = 50
batch_size_value = 5

input_video = Input(shape=(no_of_frames, img_width, img_height, channels))
encoded_sequence = ConvLSTM2D(64, kernel_size=(5, 5), strides=(1, 1),padding='same', return_sequences=False)(input_video)

GAP_layer = GlobalAveragePooling2D()(encoded_sequence)

hidden_layer_1 = Dense(activation="relu", units=1024)(GAP_layer)
drop_layer=Dropout(0.2)(hidden_layer_1)
hidden_layer_2 = Dense(activation="relu", units=512)(drop_layer)
outputs = Dense(5, activation="softmax")(hidden_layer_2)
model = Model([input_video], outputs)

model.summary()
nadam_optimizer = Nadam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)

model.compile(loss="categorical_crossentropy", optimizer=nadam_optimizer, metrics=["accuracy"])


# Save the model according to the conditions
checkpoint = ModelCheckpoint("Xception_convLSTM2D.h5", monitor='val_acc', verbose=1, save_best_only=True,
                            save_weights_only=False,
                            mode='auto', period=1)

history = model.fit(
        all_features_array['train'], all_labels_array['train'],
        validation_data=(all_features_array['validation'], all_labels_array['validation']),
        verbose=1,
        epochs=no_of_epochs,
        callbacks=[checkpoint])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 9, 7, 7, 2048)     0         
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, 7, 7, 64)          13517056  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              66560     
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 2565      
Total para

In [ ]:
prediction = model.evaluate_generator(all_features_array['test'])
print("Loss: ", prediction[0], "Accuracy: ", prediction[1])

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper left')
plt.show()


In [ ]:
path = '/home/datasets/metadata'

count = 0
for pathology in os.listdir(path):
    pathology_dir = os.path.join(path,pathology)
    for subject in os.listdir(pathology_dir):
        if subject != 's3':continue
        path2 = os.path.join(pathology_dir,subject,'silhouettes','side_view','metadata','key_frames.json')
        with open(path2) as f:
           key_frames = json.load(f)
        
        for pat in key_frames:
            print(pat)
            count += len(key_frames[pat])
print(count//9)